In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import numpy as np
import pandas as pd
df = pd.read_csv('lung cancer survey.csv')
df_no_na = df.dropna()

df_age = df_no_na[df_no_na["AGE"] > 21]
# Use df_age for all models, where clustering algorithm models, such as Kmeans, randomforest, decision tree as they would have already categorise an age threshold within the model
df_age

# Use df_cluster only for logistic regression, lasso, ridge and elastic net - cross comparison of these models with df_age
df_cluster = df_age.copy()
df_cluster['cluster'] = df_cluster['AGE'].apply(lambda x: 1 if x >= 61 else 0)
df_cluster = df_cluster.drop("AGE", axis = 1)
df_cluster

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,0.0,61.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1.0,70.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
2,1.0,59.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,54.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
4,0.0,54.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8996,1.0,62.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
8997,0.0,71.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
8998,1.0,63.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
8999,1.0,70.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0


,GENDER,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER,cluster
0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1
1,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
3,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0
4,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8996,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1
8997,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1
8998,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1
8999,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1


# Using numerical Age Column

## Simple logistics regression

In [9]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

y = df_age['LUNG_CANCER'].values
X = df_age.drop("LUNG_CANCER", axis = 1).values

# Split the data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=999)

# Initialize and fit the logistic regression model
model = LogisticRegression(penalty = None)        # By default, penalty is l2 (i.e. ridge)
model.fit(X_train, y_train)

# Print the model coefficients
model.intercept_
model.coef_

# Fitted / Predicted y values: Default Threshold is 0.5
# Fitted / Predicted y values: Default Threshold is 0.5
y_pred = model.predict(X_val)                        # Gives values either 1 or 0, dependent on whether the probability is more than 0.5 or less than 0.5
y_pred

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")

# Confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))       # Row wise is actual while column wise is predicted

# Precision, Recall, F1 Score
print(f"Precision: {precision_score(y_val, y_pred)}")
print(f"Recall: {recall_score(y_val, y_pred)}")
print(f"F1 Score: {f1_score(y_val, y_pred)}")

/Users/junhaoteo/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(penalty=None)

array([-0.57910851])

array([[-0.90549117,  0.01406197,  0.34941875,  1.37261275,  0.04109201,
         0.24159865, -0.18032971,  0.52097332,  0.7251134 , -0.69060379,
         1.49793398, -0.0251325 , -0.37964685,  0.71966336, -0.08911034]])

array([1., 1., 1., ..., 1., 1., 0.])

Accuracy: 0.875
Confusion Matrix:
 [[ 154  224]
 [   1 1421]]
Precision: 0.8638297872340426
Recall: 0.9992967651195499
F1 Score: 0.9266384088686013


In [31]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import f1_score

# Define cross-validation strategy (e.g., 5-fold cross-validation)
kf = KFold(n_splits=10, shuffle=True, random_state=999)

# Generate a large grid for C values
C_grid = np.logspace(-5, 3, num=100)  # From 10^-5 to 10^3 with 10 points

# Define parameter grids for Lasso, Ridge, and ElasticNet
lasso_param_grid = {'C': C_grid}
ridge_param_grid = {'C': C_grid}
elasticnet_param_grid = {
    'C': C_grid,
    'l1_ratio': np.linspace(0.01, 1, num=5)  # L1_ratio from 0.01 to 1 with 5 points
}

# Initialize the models
lasso_logistic_model = LogisticRegression(penalty='l1', solver='liblinear', max_iter=10000, random_state=999)
ridge_logistic_model = LogisticRegression(penalty='l2', solver='liblinear', max_iter=10000, random_state=999)
elasticnet_logistic_model = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=10000, random_state=999)

# Lasso: Perform Grid Search with Cross-Validation on the training set
lasso_logistic_grid_search = GridSearchCV(lasso_logistic_model, lasso_param_grid, cv=kf, scoring='f1')
lasso_logistic_grid_search.fit(X_train, y_train)

print("Best parameters for Lasso Logistic Regression:", lasso_logistic_grid_search.best_params_)
print(f"Best cross-validated f1-score for Lasso Logistic Regression: {lasso_logistic_grid_search.best_score_:.4f}\n")

# Ridge: Perform Grid Search with Cross-Validation on the training set
ridge_logistic_grid_search = GridSearchCV(ridge_logistic_model, ridge_param_grid, cv=kf, scoring='f1')
ridge_logistic_grid_search.fit(X_train, y_train)

print("Best parameters for Ridge Logistic Regression:", ridge_logistic_grid_search.best_params_)
print(f"Best cross-validated f1-score for Ridge Logistic Regression: {ridge_logistic_grid_search.best_score_:.4f}\n")

# Elastic Net: Perform Grid Search with Cross-Validation on the training set
elasticnet_logistic_grid_search = GridSearchCV(elasticnet_logistic_model, elasticnet_param_grid, cv=kf, scoring='f1')
elasticnet_logistic_grid_search.fit(X_train, y_train)

print("Best parameters for Elastic Net Logistic Regression:", elasticnet_logistic_grid_search.best_params_)
print(f"Best cross-validated f1-score for Elastic Net Logistic Regression: {elasticnet_logistic_grid_search.best_score_:.4f}\n")

# Retrieve best model
lasso_best_model = lasso_logistic_grid_search.best_estimator_
ridge_best_model = ridge_logistic_grid_search.best_estimator_
elasticnet_best_model = elasticnet_logistic_grid_search.best_estimator_

# Use best model to predict on the validation set
y_pred_lasso = lasso_best_model.predict(X_val)
y_pred_ridge = ridge_best_model.predict(X_val)
y_pred_elasticnet = elasticnet_best_model.predict(X_val)

# Evaluate the best models: Calculate F1-score for each best model on the validation set
print(f"Lasso Logistic Regression F1 Score on Validation Set: {f1_score(y_val, y_pred_lasso):.4f}")
print(f"Ridge Logistic Regression F1 Score on Validation Set: {f1_score(y_val, y_pred_ridge):.4f}")
print(f"Elastic Net Logistic Regression F1 Score on Validation Set: {f1_score(y_val, y_pred_elasticnet):.4f}")

GridSearchCV(cv=KFold(n_splits=10, random_state=999, shuffle=True),
             estimator=LogisticRegression(max_iter=10000, penalty='l1',
                                          random_state=999,
                                          solver='liblinear'),
             param_grid={'C': array([1.00000000e-05, 1.20450354e-05, 1.45082878e-05, 1.74752840e-05,
       2.10490414e-05, 2.53536449e-05, 3.05385551e-05, 3.67837977e-05,
       4.43062146e-05, 5.33669923e-05, 6.42807312e-05, 7.7...
       1.38488637e+01, 1.66810054e+01, 2.00923300e+01, 2.42012826e+01,
       2.91505306e+01, 3.51119173e+01, 4.22924287e+01, 5.09413801e+01,
       6.13590727e+01, 7.39072203e+01, 8.90215085e+01, 1.07226722e+02,
       1.29154967e+02, 1.55567614e+02, 1.87381742e+02, 2.25701972e+02,
       2.71858824e+02, 3.27454916e+02, 3.94420606e+02, 4.75081016e+02,
       5.72236766e+02, 6.89261210e+02, 8.30217568e+02, 1.00000000e+03])},
             scoring='f1')

Best parameters for Lasso Logistic Regression: {'C': 4.534878508128592}
Best cross-validated f1-score for Lasso Logistic Regression: 0.9325



GridSearchCV(cv=KFold(n_splits=10, random_state=999, shuffle=True),
             estimator=LogisticRegression(max_iter=10000, random_state=999,
                                          solver='liblinear'),
             param_grid={'C': array([1.00000000e-05, 1.20450354e-05, 1.45082878e-05, 1.74752840e-05,
       2.10490414e-05, 2.53536449e-05, 3.05385551e-05, 3.67837977e-05,
       4.43062146e-05, 5.33669923e-05, 6.42807312e-05, 7.74263683e-05,
       9...
       1.38488637e+01, 1.66810054e+01, 2.00923300e+01, 2.42012826e+01,
       2.91505306e+01, 3.51119173e+01, 4.22924287e+01, 5.09413801e+01,
       6.13590727e+01, 7.39072203e+01, 8.90215085e+01, 1.07226722e+02,
       1.29154967e+02, 1.55567614e+02, 1.87381742e+02, 2.25701972e+02,
       2.71858824e+02, 3.27454916e+02, 3.94420606e+02, 4.75081016e+02,
       5.72236766e+02, 6.89261210e+02, 8.30217568e+02, 1.00000000e+03])},
             scoring='f1')

Best parameters for Ridge Logistic Regression: {'C': 11.49756995397738}
Best cross-validated f1-score for Ridge Logistic Regression: 0.9324



GridSearchCV(cv=KFold(n_splits=10, random_state=999, shuffle=True),
             estimator=LogisticRegression(max_iter=10000, penalty='elasticnet',
                                          random_state=999, solver='saga'),
             param_grid={'C': array([1.00000000e-05, 1.20450354e-05, 1.45082878e-05, 1.74752840e-05,
       2.10490414e-05, 2.53536449e-05, 3.05385551e-05, 3.67837977e-05,
       4.43062146e-05, 5.33669923e-05, 6.42807312e-05,...
       2.91505306e+01, 3.51119173e+01, 4.22924287e+01, 5.09413801e+01,
       6.13590727e+01, 7.39072203e+01, 8.90215085e+01, 1.07226722e+02,
       1.29154967e+02, 1.55567614e+02, 1.87381742e+02, 2.25701972e+02,
       2.71858824e+02, 3.27454916e+02, 3.94420606e+02, 4.75081016e+02,
       5.72236766e+02, 6.89261210e+02, 8.30217568e+02, 1.00000000e+03]),
                         'l1_ratio': array([0.01  , 0.2575, 0.505 , 0.7525, 1.    ])},
             scoring='f1')

Best parameters for Elastic Net Logistic Regression: {'C': 50.94138014816386, 'l1_ratio': 1.0}
Best cross-validated f1-score for Elastic Net Logistic Regression: 0.9314

Lasso Logistic Regression F1 Score on Validation Set: 0.9263
Ridge Logistic Regression F1 Score on Validation Set: 0.9263
Elastic Net Logistic Regression F1 Score on Validation Set: 0.9240


# Using cluster, (all binary data)


## Simple logistic regression

In [76]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

y = df_cluster['LUNG_CANCER'].values
X = df_cluster.drop("LUNG_CANCER", axis = 1).values

# Split the data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=999)

# Initialize and fit the logistic regression model
model = LogisticRegression(penalty = None)        # By default, penalty is l2 (i.e. ridge)
model.fit(X_train, y_train)

# Print the model coefficients
model.intercept_
model.coef_

# Fitted / Predicted y values: Default Threshold is 0.5
y_pred = model.predict(X_val)                        # Gives values either 1 or 0, dependent on whether the probability is more than 0.5 or less than 0.5
y_pred

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")

# Confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))       # Row wise is actual while column wise is predicted

# Precision, Recall, F1 Score
print(f"Precision: {precision_score(y_val, y_pred)}")
print(f"Recall: {recall_score(y_val, y_pred)}")
print(f"F1 Score: {f1_score(y_val, y_pred)}")

LogisticRegression(penalty=None)

array([0.1456049])

array([[-0.90611442,  0.34958331,  1.37635644,  0.03894071,  0.24325014,
        -0.18102477,  0.52218976,  0.71693491, -0.69725856,  1.49781326,
        -0.02496981, -0.38448468,  0.71771285, -0.09045884,  0.27086448]])

array([1., 1., 1., ..., 1., 1., 0.])

Accuracy: 0.8677777777777778
Confusion Matrix:
 [[ 142  236]
 [   2 1420]]
Precision: 0.857487922705314
Recall: 0.9985935302390999
F1 Score: 0.9226770630279402


In [38]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import f1_score

# Define cross-validation strategy (e.g., 5-fold cross-validation)
kf = KFold(n_splits=10, shuffle=True, random_state=999)

# Generate a large grid for C values
C_grid = np.logspace(-5, 3, num=100)  # From 10^-5 to 10^3 with 10 points

# Define parameter grids for Lasso, Ridge, and ElasticNet
lasso_param_grid = {'C': C_grid}
ridge_param_grid = {'C': C_grid}
elasticnet_param_grid = {
    'C': C_grid,
    'l1_ratio': np.linspace(0.01, 1, num=5)  # L1_ratio from 0.01 to 1 with 5 points
}

# Initialize the models
lasso_logistic_model = LogisticRegression(penalty='l1', solver='liblinear', max_iter=10000, random_state=999)
ridge_logistic_model = LogisticRegression(penalty='l2', solver='liblinear', max_iter=10000, random_state=999)
elasticnet_logistic_model = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=10000, random_state=999)

# Lasso: Perform Grid Search with Cross-Validation on the training set
lasso_logistic_grid_search = GridSearchCV(lasso_logistic_model, lasso_param_grid, cv=kf, scoring='f1')
lasso_logistic_grid_search.fit(X_train, y_train)

print("Best parameters for Lasso Logistic Regression:", lasso_logistic_grid_search.best_params_)
print(f"Best cross-validated f1-score for Lasso Logistic Regression: {lasso_logistic_grid_search.best_score_:.4f}\n")

# Ridge: Perform Grid Search with Cross-Validation on the training set
ridge_logistic_grid_search = GridSearchCV(ridge_logistic_model, ridge_param_grid, cv=kf, scoring='f1')
ridge_logistic_grid_search.fit(X_train, y_train)

print("Best parameters for Ridge Logistic Regression:", ridge_logistic_grid_search.best_params_)
print(f"Best cross-validated f1-score for Ridge Logistic Regression: {ridge_logistic_grid_search.best_score_:.4f}\n")

# Elastic Net: Perform Grid Search with Cross-Validation on the training set
elasticnet_logistic_grid_search = GridSearchCV(elasticnet_logistic_model, elasticnet_param_grid, cv=kf, scoring='f1')
elasticnet_logistic_grid_search.fit(X_train, y_train)

print("Best parameters for Elastic Net Logistic Regression:", elasticnet_logistic_grid_search.best_params_)
print(f"Best cross-validated f1-score for Elastic Net Logistic Regression: {elasticnet_logistic_grid_search.best_score_:.4f}\n")

# Retrieve best model
lasso_best_model = lasso_logistic_grid_search.best_estimator_
ridge_best_model = ridge_logistic_grid_search.best_estimator_
elasticnet_best_model = elasticnet_logistic_grid_search.best_estimator_

# Use best model to predict on the validation set
y_pred_lasso = lasso_best_model.predict(X_val)
y_pred_ridge = ridge_best_model.predict(X_val)
y_pred_elasticnet = elasticnet_best_model.predict(X_val)

# Evaluate the best models: Calculate F1-score for each best model on the validation set
print(f"Lasso Logistic Regression F1 Score on Validation Set: {f1_score(y_val, y_pred_lasso):.4f}")
print(f"Ridge Logistic Regression F1 Score on Validation Set: {f1_score(y_val, y_pred_ridge):.4f}")
print(f"Elastic Net Logistic Regression F1 Score on Validation Set: {f1_score(y_val, y_pred_elasticnet):.4f}")

GridSearchCV(cv=KFold(n_splits=10, random_state=999, shuffle=True),
             estimator=LogisticRegression(max_iter=10000, penalty='l1',
                                          random_state=999,
                                          solver='liblinear'),
             param_grid={'C': array([1.00000000e-05, 1.20450354e-05, 1.45082878e-05, 1.74752840e-05,
       2.10490414e-05, 2.53536449e-05, 3.05385551e-05, 3.67837977e-05,
       4.43062146e-05, 5.33669923e-05, 6.42807312e-05, 7.7...
       1.38488637e+01, 1.66810054e+01, 2.00923300e+01, 2.42012826e+01,
       2.91505306e+01, 3.51119173e+01, 4.22924287e+01, 5.09413801e+01,
       6.13590727e+01, 7.39072203e+01, 8.90215085e+01, 1.07226722e+02,
       1.29154967e+02, 1.55567614e+02, 1.87381742e+02, 2.25701972e+02,
       2.71858824e+02, 3.27454916e+02, 3.94420606e+02, 4.75081016e+02,
       5.72236766e+02, 6.89261210e+02, 8.30217568e+02, 1.00000000e+03])},
             scoring='f1')

Best parameters for Lasso Logistic Regression: {'C': 6.5793322465756825}
Best cross-validated f1-score for Lasso Logistic Regression: 0.9319



GridSearchCV(cv=KFold(n_splits=10, random_state=999, shuffle=True),
             estimator=LogisticRegression(max_iter=10000, random_state=999,
                                          solver='liblinear'),
             param_grid={'C': array([1.00000000e-05, 1.20450354e-05, 1.45082878e-05, 1.74752840e-05,
       2.10490414e-05, 2.53536449e-05, 3.05385551e-05, 3.67837977e-05,
       4.43062146e-05, 5.33669923e-05, 6.42807312e-05, 7.74263683e-05,
       9...
       1.38488637e+01, 1.66810054e+01, 2.00923300e+01, 2.42012826e+01,
       2.91505306e+01, 3.51119173e+01, 4.22924287e+01, 5.09413801e+01,
       6.13590727e+01, 7.39072203e+01, 8.90215085e+01, 1.07226722e+02,
       1.29154967e+02, 1.55567614e+02, 1.87381742e+02, 2.25701972e+02,
       2.71858824e+02, 3.27454916e+02, 3.94420606e+02, 4.75081016e+02,
       5.72236766e+02, 6.89261210e+02, 8.30217568e+02, 1.00000000e+03])},
             scoring='f1')

Best parameters for Ridge Logistic Regression: {'C': 3.7649358067924714}
Best cross-validated f1-score for Ridge Logistic Regression: 0.9319



GridSearchCV(cv=KFold(n_splits=10, random_state=999, shuffle=True),
             estimator=LogisticRegression(max_iter=10000, penalty='elasticnet',
                                          random_state=999, solver='saga'),
             param_grid={'C': array([1.00000000e-05, 1.20450354e-05, 1.45082878e-05, 1.74752840e-05,
       2.10490414e-05, 2.53536449e-05, 3.05385551e-05, 3.67837977e-05,
       4.43062146e-05, 5.33669923e-05, 6.42807312e-05,...
       2.91505306e+01, 3.51119173e+01, 4.22924287e+01, 5.09413801e+01,
       6.13590727e+01, 7.39072203e+01, 8.90215085e+01, 1.07226722e+02,
       1.29154967e+02, 1.55567614e+02, 1.87381742e+02, 2.25701972e+02,
       2.71858824e+02, 3.27454916e+02, 3.94420606e+02, 4.75081016e+02,
       5.72236766e+02, 6.89261210e+02, 8.30217568e+02, 1.00000000e+03]),
                         'l1_ratio': array([0.01  , 0.2575, 0.505 , 0.7525, 1.    ])},
             scoring='f1')

Best parameters for Elastic Net Logistic Regression: {'C': 4.534878508128592, 'l1_ratio': 0.01}
Best cross-validated f1-score for Elastic Net Logistic Regression: 0.9319

Lasso Logistic Regression F1 Score on Validation Set: 0.9227
Ridge Logistic Regression F1 Score on Validation Set: 0.9227
Elastic Net Logistic Regression F1 Score on Validation Set: 0.9227


# Inclusion of pairwise features

In [5]:
import pandas as pd
from itertools import product

feature_names = [col for col in df_cluster.columns if col != "LUNG_CANCER"]


# Generate all combinations of interaction terms for these features
for i in range(len(feature_names)):
    for j in range(i + 1, len(feature_names)):
        new_column_name = f'{feature_names[i]}_{feature_names[j]}'
        df_cluster[new_column_name] = df_cluster[feature_names[i]] * df_cluster[feature_names[j]]

print(df_cluster)

      GENDER  SMOKING  YELLOW_FINGERS  ANXIETY  PEER_PRESSURE  \
0        0.0      0.0             0.0      1.0            1.0   
1        1.0      1.0             1.0      0.0            0.0   
2        1.0      0.0             0.0      0.0            0.0   
3        1.0      0.0             0.0      0.0            1.0   
4        0.0      1.0             0.0      0.0            1.0   
...      ...      ...             ...      ...            ...   
8996     1.0      0.0             1.0      1.0            1.0   
8997     0.0      1.0             1.0      1.0            0.0   
8998     1.0      1.0             0.0      0.0            1.0   
8999     1.0      1.0             1.0      0.0            0.0   
9099     1.0      0.0             0.0      0.0            1.0   

      CHRONIC DISEASE  FATIGUE   ALLERGY   WHEEZING  ALCOHOL CONSUMING  ...  \
0                 0.0       1.0       0.0       0.0                0.0  ...   
1                 1.0       0.0       1.0       1.0          

/var/folders/fc/f1h6qsms6kn106201z7qmqdc0000gn/T/ipykernel_75326/1769844033.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cluster[new_column_name] = df_cluster[feature_names[i]] * df_cluster[feature_names[j]]
/var/folders/fc/f1h6qsms6kn106201z7qmqdc0000gn/T/ipykernel_75326/1769844033.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cluster[new_column_name] = df_cluster[feature_names[i]] * df_cluster[feature_names[j]]
/var/folders/fc/f1h6qsms6kn106201z7qmqdc0000gn/T/ipykernel_75326/1769844033.py:11: PerformanceWa

In [11]:
y = df_cluster['LUNG_CANCER'].values
X = df_cluster.drop("LUNG_CANCER", axis = 1).values

# Split the data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=999)

# Initialize and fit the logistic regression model
model = LogisticRegression(penalty = None)        # By default, penalty is l2 (i.e. ridge)
model.fit(X_train, y_train)

# Fitted / Predicted y values: Default Threshold is 0.5
y_pred = model.predict(X_val)                        # Gives values either 1 or 0, dependent on whether the probability is more than 0.5 or less than 0.5
y_pred

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_val, y_pred)}")

# Confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))       # Row wise is actual while column wise is predicted

# Precision, Recall, F1 Score
print(f"Precision: {precision_score(y_val, y_pred)}")
print(f"Recall: {recall_score(y_val, y_pred)}")
print(f"F1 Score: {f1_score(y_val, y_pred)}")

LogisticRegression(penalty=None)

array([0., 1., 1., ..., 0., 1., 0.])

Accuracy: 0.8827777777777778
Confusion Matrix:
 [[ 193  185]
 [  26 1396]]
Precision: 0.8829854522454142
Recall: 0.9817158931082982
F1 Score: 0.9297369297369298


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import f1_score

# Define cross-validation strategy (e.g., 10-fold cross-validation)
kf = KFold(n_splits=10, shuffle=True, random_state=999)

# Generate a large grid for C values
C_grid = np.logspace(-5, 5, num=150)  

# Define parameter grids for Lasso, Ridge, and ElasticNet
lasso_param_grid = {'C': C_grid}
ridge_param_grid = {'C': C_grid}

# Initialize the models
lasso_logistic_model = LogisticRegression(penalty='l1', solver='liblinear', max_iter=10000, random_state=999)
ridge_logistic_model = LogisticRegression(penalty='l2', solver='liblinear', max_iter=10000, random_state=999)

# Lasso: Perform Grid Search with Cross-Validation on the training set
lasso_logistic_grid_search = GridSearchCV(lasso_logistic_model, lasso_param_grid, cv=kf, scoring='f1')
lasso_logistic_grid_search.fit(X_train, y_train)

print("Best parameters for Lasso Logistic Regression:", lasso_logistic_grid_search.best_params_)
print(f"Best cross-validated f1-score for Lasso Logistic Regression: {lasso_logistic_grid_search.best_score_:.4f}\n")

# Ridge: Perform Grid Search with Cross-Validation on the training set
ridge_logistic_grid_search = GridSearchCV(ridge_logistic_model, ridge_param_grid, cv=kf, scoring='f1')
ridge_logistic_grid_search.fit(X_train, y_train)

print("Best parameters for Ridge Logistic Regression:", ridge_logistic_grid_search.best_params_)
print(f"Best cross-validated f1-score for Ridge Logistic Regression: {ridge_logistic_grid_search.best_score_:.4f}\n")


# Retrieve best model
lasso_best_model = lasso_logistic_grid_search.best_estimator_
ridge_best_model = ridge_logistic_grid_search.best_estimator_


# Use best model to predict on the validation set
y_pred_lasso = lasso_best_model.predict(X_val)
y_pred_ridge = ridge_best_model.predict(X_val)


# Evaluate the best models: Calculate F1-score for each best model on the validation set
print(f"Lasso Logistic Regression F1 Score on Validation Set: {f1_score(y_val, y_pred_lasso):.4f}")
print(f"Ridge Logistic Regression F1 Score on Validation Set: {f1_score(y_val, y_pred_ridge):.4f}")


GridSearchCV(cv=KFold(n_splits=10, random_state=999, shuffle=True),
             estimator=LogisticRegression(max_iter=10000, penalty='l1',
                                          random_state=999,
                                          solver='liblinear'),
             param_grid={'C': array([1.00000000e-05, 1.16711619e-05, 1.36216020e-05, 1.58979923e-05,
       1.85548042e-05, 2.16556124e-05, 2.52746159e-05, 2.94984134e-05,
       3.44280759e-05, 4.01815648e-05, 4.68965549e-05, 5.4...
       2.09965360e+03, 2.45053971e+03, 2.86006458e+03, 3.33802767e+03,
       3.89586614e+03, 4.54692846e+03, 5.30679382e+03, 6.19364499e+03,
       7.22870335e+03, 8.43673672e+03, 9.84665203e+03, 1.14921870e+04,
       1.34127175e+04, 1.56541998e+04, 1.82702700e+04, 2.13235280e+04,
       2.48870348e+04, 2.90460612e+04, 3.39001283e+04, 3.95653887e+04,
       4.61774057e+04, 5.38943979e+04, 6.29010244e+04, 7.34128040e+04,
       8.56812721e+04, 1.00000000e+05])},
             scoring='f1')

Best parameters for Lasso Logistic Regression: {'C': 0.49886906858441177}
Best cross-validated f1-score for Lasso Logistic Regression: 0.9365



GridSearchCV(cv=KFold(n_splits=10, random_state=999, shuffle=True),
             estimator=LogisticRegression(max_iter=10000, random_state=999,
                                          solver='liblinear'),
             param_grid={'C': array([1.00000000e-05, 1.16711619e-05, 1.36216020e-05, 1.58979923e-05,
       1.85548042e-05, 2.16556124e-05, 2.52746159e-05, 2.94984134e-05,
       3.44280759e-05, 4.01815648e-05, 4.68965549e-05, 5.47337285e-05,
       6...
       2.09965360e+03, 2.45053971e+03, 2.86006458e+03, 3.33802767e+03,
       3.89586614e+03, 4.54692846e+03, 5.30679382e+03, 6.19364499e+03,
       7.22870335e+03, 8.43673672e+03, 9.84665203e+03, 1.14921870e+04,
       1.34127175e+04, 1.56541998e+04, 1.82702700e+04, 2.13235280e+04,
       2.48870348e+04, 2.90460612e+04, 3.39001283e+04, 3.95653887e+04,
       4.61774057e+04, 5.38943979e+04, 6.29010244e+04, 7.34128040e+04,
       8.56812721e+04, 1.00000000e+05])},
             scoring='f1')

Best parameters for Ridge Logistic Regression: {'C': 0.793101660333306}
Best cross-validated f1-score for Ridge Logistic Regression: 0.9364



In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import f1_score
# Define cross-validation strategy (e.g., 10-fold cross-validation)
kf = KFold(n_splits=10, shuffle=True, random_state=999)

# Generate a large grid for C values
C_grid = np.logspace(-5, 5, num=150)  
elasticnet_param_grid = {
    'C': C_grid,
    'l1_ratio': np.linspace(0.01, 1, num=10)  # L1_ratio from 0.01 to 1 with 10 points
}

elasticnet_logistic_model = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=10000, random_state=999)

# Elastic Net: Perform Grid Search with Cross-Validation on the training set
elasticnet_logistic_grid_search = GridSearchCV(elasticnet_logistic_model, elasticnet_param_grid, cv=kf, scoring='f1')
elasticnet_logistic_grid_search.fit(X_train, y_train)

print("Best parameters for Elastic Net Logistic Regression:", elasticnet_logistic_grid_search.best_params_)
print(f"Best cross-validated f1-score for Elastic Net Logistic Regression: {elasticnet_logistic_grid_search.best_score_:.4f}\n")

elasticnet_best_model = elasticnet_logistic_grid_search.best_estimator_
y_pred_elasticnet = elasticnet_best_model.predict(X_val)
print(f"Elastic Net Logistic Regression F1 Score on Validation Set: {f1_score(y_val, y_pred_elasticnet):.4f}")